# Tidy Data in Python
by [Jean-Nicholas Hould](http://www.jeannicholashould.com/)


from the blog post of the same name
[http://www.jeannicholashould.com/tidy-data-in-python.html](http://www.jeannicholashould.com/tidy-data-in-python.html)

## Tidying messy datasets

The five most common problems with messy datasets are:

- Column headers are values, not variable names
- Multiple variables are stored in on column
- Variables are stored in both rows and columns
- Multiple types of observational units are stored in the same table
- A single observational unit is stored in multiple tables

We'll run through how to fix all of these types of problems in the examples below

---


In [1]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

## Column headers are values, not variable names

### Pew Research Center Dataset

This dataset explores the relationship between income and religion.

Problem: 

- The columns headers are composed of the possible income values.

In [2]:
df = pd.read_csv("../tidy-data-python/data/pew-raw.csv")
df

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,244,236,238,197,223
8,Jehovahs Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


In [3]:
formatted_df = pd.melt(df,["religion"], var_name="income", value_name="freq")
formatted_df = formatted_df.sort_values(by=["religion"])
formatted_df.head(15)

,religion,income,freq
0,Agnostic,<$10k,27
30,Agnostic,$30-40k,81
40,Agnostic,$40-50k,76
50,Agnostic,$50-75k,137
10,Agnostic,$10-20k,34
20,Agnostic,$20-30k,60
41,Atheist,$40-50k,35
21,Atheist,$20-30k,37
11,Atheist,$10-20k,27
31,Atheist,$30-40k,52


### Billboard Top 100 Dataset

In [4]:
df = pd.read_csv("./data/billboard.csv", encoding="mac_latin2")
df.head(10)

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17,2000-08-26,59,52.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25,2000-03-18,83,83.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01,2000-06-24,63,45.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2000,Sisqo,Incomplete,3:52,Rock,2000-06-24,2000-08-12,77,66.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,81,54.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Melting, or "unpivoting"

# Need to specify the columns you don't want to unpivot, called the id_variables,
# and the names of the new columns
id_vars = ["year","artist.inverted","track","time","genre","date.entered","date.peaked"]
df = pd.melt(frame=df, id_vars=id_vars, var_name="week", value_name="rank")

df.head(10)

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,x1st.week,57.0
5,2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17,2000-08-26,x1st.week,59.0
6,2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25,2000-03-18,x1st.week,83.0
7,2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01,2000-06-24,x1st.week,63.0
8,2000,Sisqo,Incomplete,3:52,Rock,2000-06-24,2000-08-12,x1st.week,77.0
9,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,x1st.week,81.0


In [6]:
# Cleaning out unnecessary rows before formatting numbers properly
# since weeks where the track wasn't on the chart were filled with NAs
df = df.dropna()

# Formatting 
df["week"] = df['week'].str.extract('(\d+)', expand=False).astype(int)
df["rank"] = df["rank"].astype(int)

# Create "date" columns
df['date'] = pd.to_datetime(df['date.entered']) + pd.to_timedelta(df['week'], unit='w') - pd.DateOffset(weeks=1)

df.head(10)

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank,date
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78,2000-09-23
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15,2000-02-12
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71,1999-10-23
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41,2000-08-12
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57,2000-08-05
5,2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17,2000-08-26,1,59,2000-06-17
6,2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25,2000-03-18,1,83,1999-12-25
7,2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01,2000-06-24,1,63,2000-04-01
8,2000,Sisqo,Incomplete,3:52,Rock,2000-06-24,2000-08-12,1,77,2000-06-24
9,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,1,81,1999-06-05


In [7]:
# Only keeping interesting columns
df = df[["year", "artist.inverted", "track", "time", "genre", "week", "rank", "date"]]

# Reassign after sort rather than sort in place
df = df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

# Assigning the tidy dataset to a variable for future usage
billboard = df

billboard.head(10)

,year,artist.inverted,track,time,genre,week,rank,date
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87,2000-02-26
563,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82,2000-03-04
880,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72,2000-03-11
1197,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77,2000-03-18
1514,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87,2000-03-25
1831,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,6,94,2000-04-01
2148,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,7,99,2000-04-08
287,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1,91,2000-09-02
604,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,2,87,2000-09-09
921,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,3,92,2000-09-16


A tidier version of the dataset is shown below. There is still a lot of repetition of the song details: the track name, time and genre. For this reason, this dataset is still not completely tidy as per Wickham’s definition. We will address this in the next example.

## Multiple types in one table

Following up on the Billboard dataset, we’ll now address the repetition problem of the previous table.

Problem:

* Multiple observational units (the `song` and its `rank`) in a single table.

Solution:

* Create a new `songs` table with a primary key, `song_id`
* Join the `songs` table with the original `billboard` table to transfer over the `song_id` which acts as a foreign key, then only keep the columns related to the song rankings

We’ll first create a `songs` table which contains the details of each song:

In [8]:
# Keep only the columns from the `billboard` table which are attributes of the songs 
# themselves, not anything about their rankings
songs_cols = ["year", "artist.inverted", "track", "time", "genre"]

# Drop all the duplicate rows which used to be separate weeks in the rankings
songs = billboard[songs_cols].drop_duplicates()

# Reindex that new table so each row gets an integer index in order
songs = songs.reset_index(drop=True)

# Create a new `song_id` column 
songs["song_id"] = songs.index

songs.head(10)

,year,artist.inverted,track,time,genre,song_id
0,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,0
1,2000,2Ge+her,The Hardest Part Of Breaking Up (Is Getting Ba...,3:15,R&B,1
2,2000,3 Doors Down,Kryptonite,3:53,Rock,2
3,2000,3 Doors Down,Loser,4:24,Rock,3
4,2000,504 Boyz,Wobble Wobble,3:35,Rap,4
5,2000,98°,Give Me Just One Night (Una Noche),3:24,Rock,5
6,2000,A*Teens,Dancing Queen,3:44,Pop,6
7,2000,Aaliyah,I Don't Wanna,4:15,Rock,7
8,2000,Aaliyah,Try Again,4:03,Rock,8
9,2000,"Adams, Yolanda",Open My Heart,5:30,Gospel,9


We’ll then create a `ranks` table which only contains the `song_id`, `date` and the `rank`.

Doing a "merge" here between `billboard` and `songs` and then in the end only keeping three of the columns seems like a lot of extra overhead or wasted work, but it's a foolproof way to get the `song_id` foreign key associations correct in the new `ranks` table.

In [9]:
ranks = pd.merge(billboard, songs, on=["year","artist.inverted", "track", "time", "genre"])
ranks = ranks[["song_id", "date","rank"]]
ranks.head(10)

,song_id,date,rank
0,0,2000-02-26,87
1,0,2000-03-04,82
2,0,2000-03-11,72
3,0,2000-03-18,77
4,0,2000-03-25,87
5,0,2000-04-01,94
6,0,2000-04-08,99
7,1,2000-09-02,91
8,1,2000-09-09,87
9,1,2000-09-16,92


## Multiple variables stored in one column

### Tubercolosis Example

A few notes on the raw data set:

- The columns starting with "m" or "f" contain multiple variables: 
    - Sex ("m" or "f")
    - Age Group ("0-14","15-24", "25-34", "45-54", "55-64", "65", "unknown")
- Mixture of 0s and missing values("NaN"). This is due to the data collection process and the distinction is important for this dataset.

In order to tidy this dataset, we need to remove the different values from the header and unpivot them into rows. We’ll first need to melt the `sex + age` group columns into a single one. Once we have that single column, we’ll derive three columns from it: `sex`, `age_lower` and `age_upper`. With those, we’ll be able to properly build a tidy dataset.

In [10]:
df = pd.read_csv("./data/tb-raw.csv")
df

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,NaN,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,NaN,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,NaN,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,NaN,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


In [11]:
df = pd.melt(df, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")
df.head(20)

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0
5,AM,2000,m014,2.0
6,AN,2000,m014,0.0
7,AO,2000,m014,186.0
8,AR,2000,m014,97.0
9,AS,2000,m014,NaN


In [12]:
# Extract Sex, Age lower bound and Age upper bound group
# Regex pattern: Match a non-digit character at the beginning
#                Match exactly two digit characters at the end
#                Match one or more digits in between
tmp_df = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", expand=False)    

# Name columns
tmp_df.columns = ["sex", "age_lower", "age_upper"]

# Create `age` range column based on `age_lower` and `age_upper`
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

# Concatenate the original dataframe with this new dataframe
# axis=0 is down, axis=1 is across
df = pd.concat([df, tmp_df], axis=1)

# Drop unnecessary columns
df = df.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
# Drop unnecessary rows
df = df.dropna()
# Sort
df = df.sort_values(ascending=True,by=["country", "year", "sex", "age"])
df.head(10)

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54
50,AD,2000,0.0,m,55-64
81,AE,2000,3.0,f,0-14
1,AE,2000,2.0,m,0-14
11,AE,2000,4.0,m,15-24
21,AE,2000,4.0,m,25-34


## Variables are stored in both rows and columns

### Global Historical Climatology Network Dataset

This dataset represents the daily weather records for a weather station (MX17004) in Mexico for five months in 2010.

Problem:

- Variables are stored in both rows (tmin, tmax) and columns (days).

In order to make this dataset tidy, we want to move the three misplaced variables as three individual columns: `tmin`, `tmax` and `date`.


In [13]:
df = pd.read_csv("./data/weather-raw.csv")
df

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MX17004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX17004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MX17004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df = pd.melt(df, id_vars=["id", "year","month","element"], var_name="day_raw")
df.head(10)

,id,year,month,element,day_raw,value
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN
5,MX17004,2010,3,tmin,d1,NaN
6,MX17004,2010,4,tmax,d1,NaN
7,MX17004,2010,4,tmin,d1,NaN
8,MX17004,2010,5,tmax,d1,NaN
9,MX17004,2010,5,tmin,d1,NaN


In [15]:
# There is also a small problem where some of the IDs have a space afterwards
set(df.id)

{'MX17004', 'MX17004 '}

In [16]:
# Clean by assigning the same value to all rows
df["id"] = "MX17004"

In [17]:
# Extracting day
# Regex pattern finds the first d, but only captures the digits after that
df["day"] = df["day_raw"].str.extract("d(\d+)", expand=False)  

df.head(10)

,id,year,month,element,day_raw,value,day
0,MX17004,2010,1,tmax,d1,NaN,1
1,MX17004,2010,1,tmin,d1,NaN,1
2,MX17004,2010,2,tmax,d1,NaN,1
3,MX17004,2010,2,tmin,d1,NaN,1
4,MX17004,2010,3,tmax,d1,NaN,1
5,MX17004,2010,3,tmin,d1,NaN,1
6,MX17004,2010,4,tmax,d1,NaN,1
7,MX17004,2010,4,tmin,d1,NaN,1
8,MX17004,2010,5,tmax,d1,NaN,1
9,MX17004,2010,5,tmin,d1,NaN,1


In [18]:
df.dtypes

id          object
year         int64
month        int64
element     object
day_raw     object
value      float64
day         object
dtype: object

In [19]:
# To numeric values (original conversion)
# df[["year","month","day"]] = df[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))

# To integers
df[["year","month","day"]] = df[["year","month","day"]].astype(int)
df.dtypes

id          object
year         int64
month        int64
element     object
day_raw     object
value      float64
day          int64
dtype: object

In [20]:
# Creating a date from the different columns
df["date"] = pd.to_datetime(df[["year", "month", "day"]])
df.head(10)

,id,year,month,element,day_raw,value,day,date
0,MX17004,2010,1,tmax,d1,NaN,1,2010-01-01
1,MX17004,2010,1,tmin,d1,NaN,1,2010-01-01
2,MX17004,2010,2,tmax,d1,NaN,1,2010-02-01
3,MX17004,2010,2,tmin,d1,NaN,1,2010-02-01
4,MX17004,2010,3,tmax,d1,NaN,1,2010-03-01
5,MX17004,2010,3,tmin,d1,NaN,1,2010-03-01
6,MX17004,2010,4,tmax,d1,NaN,1,2010-04-01
7,MX17004,2010,4,tmin,d1,NaN,1,2010-04-01
8,MX17004,2010,5,tmax,d1,NaN,1,2010-05-01
9,MX17004,2010,5,tmin,d1,NaN,1,2010-05-01


In [21]:
df = df.drop(['year',"month","day", "day_raw"], axis=1)
df = df.dropna()
df

,id,element,value,date
12,MX17004,tmax,27.3,2010-02-02
13,MX17004,tmin,14.4,2010-02-02
22,MX17004,tmax,24.1,2010-02-03
23,MX17004,tmin,14.4,2010-02-03
44,MX17004,tmax,32.1,2010-03-05
45,MX17004,tmin,14.2,2010-03-05


In [22]:
# Unmelting column "element"
df = df.pivot_table(index=["id","date"], columns="element", values="value")
df.reset_index(drop=False, inplace=True)
df

element,id,date,tmax,tmin
0,MX17004,2010-02-02,27.3,14.4
1,MX17004,2010-02-03,24.1,14.4
2,MX17004,2010-03-05,32.1,14.2


## One type in multiple tables

### Baby Names in North Carolina and Minnesota

Dataset: Top 10 male and female baby names in MN and NC for 1916 & 2016

Problems:

- The data is spread across multiple tables/files.
- The “Year” and "State" variables are present in the file name.

In order to load those different files into a single DataFrame, we can run a custom script that will append the files together. Furthermore, we’ll need to extract the “Year” and "State" variables from the file name.

In [23]:
path = './data'
allFiles = glob.glob(path + "/*_baby_names_*.csv")
allFiles

['./data/MN_baby_names_1916.csv',
 './data/MN_baby_names_2016.csv',
 './data/NC_baby_names_1916.csv',
 './data/NC_baby_names_2016.csv']

In [24]:
def extract_variables_from_filename(string):
    match = re.match(".*([A-Z]{2})_baby_names_(\d{4}).*", string) 
    if match != None: 
        return {"state":match.group(1), "year":match.group(2)}
    
frame = pd.DataFrame()
df_list= []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=0)
    df.columns = map(str.lower, df.columns)
    variables_dict = extract_variables_from_filename(file_)
    df["year"] = variables_dict["year"]
    df["state"] = variables_dict["state"]
    df_list.append(df)

# axis=0 is the default for pd.concat()
df = pd.concat(df_list)
df.head(10)

,sex,name,births,year,state
0,F,Helen,817,1916,MN
1,F,Mary,810,1916,MN
2,F,Dorothy,700,1916,MN
3,F,Margaret,676,1916,MN
4,F,Ruth,585,1916,MN
5,F,Evelyn,451,1916,MN
6,F,Florence,365,1916,MN
7,F,Mildred,352,1916,MN
8,F,Alice,347,1916,MN
9,F,Irene,305,1916,MN
